In [2]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 5.0MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [3]:
!pip install eli5

     |████████████████████████████████| 112kB 5.1MB/s 


In [4]:
!pip install xgboost

In [5]:
!pip install hyperopt

In [6]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mea
from sklearn.model_selection import cross_val_score

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [7]:
cd "/content/drive/My Drive/Colab Notebooks/DW_matrix/matrix_two/dw_matrix_car"

[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/DW_matrix/matrix_two/dw_matrix_car'
/content


In [9]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [10]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

#Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list):continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [12]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring="neg_mean_absolute_error")
  return np.mean(scores), np.std(scores)

In [0]:
df['param_rok_produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split(' ')[0] ) )
#df['param_pojemność_skokowa'] = df['param_pojemność_skokowa'].map(lambda x: -1 if str(x) == 'None' else int( str(x).split('cm')[0].replace(' ','')  ))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int( str(x).split('cm')[0].replace(' ','') ))

In [15]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat',
         'param_skrzynia-biegów__cat','param_faktura-vat__cat',
         'param_moc','param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat','param_pojemność-skokowa','seller_name__cat',
         'feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat',
         'param_wersja__cat','param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']

xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}

run_model(xgb.XGBRegressor(**xgb_params), feats)

[19:45:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:45:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:45:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9560.46429774341, 98.48718139867199)

#Hyperopt

Przy wykorzystaniu hyperopt'a, wykonywana jest minimalizacja błędu.

In [16]:
def obj_func(params):
  print("training with params: ")
  print(params)
  
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

#space
xgb_reg_params = {
      'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
      'max_depth':     hp.choice('max_depth',     np.arange(5,16,1, dtype=int)),
      'subsample':     hp.quniform('subsample', 0.5, 1, 0.05),
      'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
      'objective':     'reg:squarederror',
      'n_estimators':  100,
      'seed': 0,
  }

#run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.3, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.5}
training with params: 
{'colsample_bytree': 0.55, 'learning_rate': 0.25, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9500000000000001}
training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.25, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.15000000000000002, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 

{'colsample_bytree': 0.7000000000000001,
 'learning_rate': 2,
 'max_depth': 10,
 'subsample': 1.0}

In [17]:
!git commit


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@939a36d774cb.(none)')


In [0]:
!git config --global user.email "marmiga@hotamil.com"
!git config --global user.name "Marek"

In [19]:
!git add day5.ipynb
!git commit -m 'Hyperopt'

fatal: pathspec 'day5.ipynb' did not match any files
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   day2_visualisation.ipynb
	modified:   day3_simple_model.ipynb
	modified:   day4.ipynb

Untracked files:
	data/car.h5

no changes added to commit


In [20]:
!git push -u origin master

Branch 'master' set up to track remote branch 'master' from 'origin'.
Everything up-to-date


In [21]:
!git add day5.ipynb
!git commit -m 'Hyperopt'

fatal: pathspec 'day5.ipynb' did not match any files
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   day2_visualisation.ipynb
	modified:   day3_simple_model.ipynb
	modified:   day4.ipynb

Untracked files:
	data/car.h5

no changes added to commit


In [22]:
!git push -u origin master

Branch 'master' set up to track remote branch 'master' from 'origin'.
Everything up-to-date


In [23]:
!git add day5.ipynb
!git commit -m 'Hyperopt'

fatal: pathspec 'day5.ipynb' did not match any files
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   day2_visualisation.ipynb
	modified:   day3_simple_model.ipynb
	modified:   day4.ipynb

Untracked files:
	data/car.h5

no changes added to commit


In [28]:
!git commit

hint: Waiting for your editor to close the file... error: unable to start editor 'editor'
Please supply the message using either -m or -F option.


In [25]:
!pwd

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [0]:
!git add day5.ipynb